In [1]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in nyseList:
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if index == '2022-01-10':
                                print(element.holding[j][0],close)
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    print(transactionCopy.iloc[-1]['total'])
    transactionCopy.to_csv("transaction/transaction167.csv")
    transactionCopy.to_pickle("transaction/transaction167.pkl")
    return transactionCopy.iloc[-1]['total']
evalOneMax([[0.19078223936143746, 0.986675953424998, 0.07057270627935228, 0.9528588558753942, 1.0282875027195482, 1.251806657541451, 0.823960510068463, 1.1414927855254375]])

[0.19078223936143746, 0.986675953424998, 0.07057270627935228, 0.9528588558753942, 1.0282875027195482, 1.251806657541451, 0.823960510068463, 1.1414927855254375]


In [1]:
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("result.csv")
from stageOld import getFullDfOld
getFullDfOld("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("resultOld.csv")

In [1]:
# from stage import getStage
# getStage("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544])
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("result.csv")

2000-02-28 00:00:00 0.888392984867096
0.888392984867096
2000-03-20 00:00:00 1.1160709857940674
1.1160709857940674


In [6]:
from stage import fullPrint
fullPrint(pd.read_pickle("stockData/nyseNasdaq/FIV.pkl"))

               open     high      low    close  adjclose     volume ticker  \
2016-12-19  10.0000  10.0300  10.0000  10.0300  8.355591  1509595.0    FIV   
2016-12-26   9.8300  10.1200   9.8300  10.1200  8.430565   349301.0    FIV   
2017-01-02   9.8200  10.2000   9.8200  10.1200  8.430565   338897.0    FIV   
2017-01-09  10.1400  10.2400  10.0500  10.2300  8.522197   257924.0    FIV   
2017-01-16   9.8000  10.2900   9.8000  10.2900  8.572182   251591.0    FIV   
2017-01-23  10.2000  10.2000  10.0000  10.0600  8.380584   489117.0    FIV   
2017-01-30  10.0510  10.1300  10.0000  10.0400  8.363917   455712.0    FIV   
2017-02-06  10.0201  10.1800  10.0200  10.1800  8.480546   403611.0    FIV   
2017-02-13  10.1100  10.1700  10.0300  10.0600  8.380584   263460.0    FIV   
2017-02-20   9.8100  10.1000   9.8100  10.0900  8.405569   175450.0    FIV   
2017-02-27  10.0900  10.2300  10.0700  10.1000  8.413902   331039.0    FIV   
2017-03-06  10.1400  10.1500   9.9100   9.9300  8.306475   54921